In [97]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from datetime import datetime
import re
import ast

# from pandas_profiling import ProfileReport

import my_functions
import importlib
importlib.reload(my_functions)

import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from scipy import stats


In [ ]:
RANDOM_SEED = 123

In [ ]:
input_folder = 'input//'
print(os.listdir("input"))


# DATA
### main_task.csv - оригинальные данные
### kaggle_task.csv - данные добавленные для соревнование на площаке kaggle для тестирования модели
### sample_submission.csv - результаты для проверки модели

In [ ]:
data = pd.read_csv(input_folder + 'main_task.csv')
data_kaggle = pd.read_csv(input_folder + 'kaggle_task.csv')
sample_submission = pd.read_csv(input_folder + '/sample_submission.csv')

In [ ]:
# дря корректной обработки признаков объединяем трейн и тест в один датасет
data['sample'] = 1 # помечаем где у нас трейн
data_kaggle['sample'] = 0 # помечаем где у нас тест
data_kaggle['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = data_kaggle.append(data, sort=False).reset_index(drop=True) # объединяем

## Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:
* Restaurant_id — идентификационный номер ресторана / сети ресторанов;
* City — город, в котором находится ресторан;
* Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
* Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
* Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
* Price Range — диапазон цен в ресторане;
* Number of Reviews — количество отзывов о ресторане;
* Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
* URL_TA — URL страницы ресторана на TripAdvosor;
* ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [ ]:
data.columns = data.columns.str.lower()
data.columns = [name.replace(' ', '_') for name in data.columns]
data.info()


# Обработка NaN

In [ ]:
data['cuisine_style'] = data['cuisine_style'].fillna('[\'not_specified\']')
data['Number_of_Reviews_isNAN'] = pd.isna(data.number_of_reviews).astype('uint8')
data['number_of_reviews'] = data.groupby(by='city').number_of_reviews.apply(lambda x: x.fillna(round(x.mean())))
data['price_range'].fillna(data.price_range.mode()[0], inplace=True)
data.reviews.fillna("[[], []]",inplace=True)
data.info()


# Предобработка

In [ ]:
data.drop(['restaurant_id', 'url_ta', 'id_ta'], inplace=True,  axis = 1, errors='ignore')
data['cuisine_style'] = data['cuisine_style'].apply(ast.literal_eval)
data['reviews'] = data['reviews'].apply(lambda x: re.sub((r'\bnan\b'), '\'empty_voice\'', x))
data['reviews'] = data['reviews'].apply(ast.literal_eval)

# Выбросы

In [ ]:
data.ranking.hist(bins=100)

# Обогащаем новыми признаками 

In [ ]:


data['reviews_text'] =  data['reviews'].apply(lambda x: x[0])
data['reviews_dates'] = data['reviews'].apply(lambda x: [datetime.strptime(date, '%m/%d/%Y').date() for date in x[1]])
data['price_range_int'] = data.price_range.apply(lambda x: 1 if x == '$' else 
                                                              (2 if x == '$$ - $$$' else 3))
data['review_text_tone_coef'] = data['reviews_text'].apply(lambda x: my_functions.review_text_tone(x))

set_dates = set()
data['dif_days'] = data.reviews_dates.apply(lambda x: (x[0] - x[-1]) if len(x) > 0 else pd.Timedelta('0 days') ).dt.days

last_date = set()
data.reviews_dates.apply(lambda x: last_date.update(x))
data['diff_last_date'] = data.reviews_dates.apply(my_functions.diff_today)


cuisine_style_set = set()
data.cuisine_style.apply(lambda x: cuisine_style_set.update(x))
cuisine_style_list = list(cuisine_style_set)
for cuisine in cuisine_style_list:
    data[cuisine] = data.cuisine_style.apply(lambda x: 1 if cuisine in x else 0)

data['city_orig'] = data['city'] 
data = pd.get_dummies(data, columns=['city_orig'], dummy_na=True, drop_first=False)

In [ ]:
# date_set = set()
# data.reviews_dates.apply(lambda x: date_set.update(x))
# last_date = max(date_set)
# data['diff_last_date'] = data.reviews_dates.apply(lambda x: (max([(x_i - last_date).days for x_i in x ]) if len(x) > 0 else pd.Timedelta('0 days').days ))
# data['diff_last_date']


# data.city.apply(lambda x: len(data[data.city == x]))

city_dict = data.groupby(by='city').city.size()
# data['quan_in_city'] = data.city.apply(lambda x: len(data[data.city == x]))
data['ranking_for_city'] =  data['ranking']  - data.city.apply(lambda x: city_dict[x])


## Нормируем признаки

In [ ]:
data['ranking_norm_by_city'] = data.groupby('city').ranking.transform(lambda x: (x - x.mean()) / x.std())
data['number_of_reviews_norm'] = data.number_of_reviews.apply(lambda x: (x - data.number_of_reviews.mean()) / data.number_of_reviews.std())

In [ ]:
# data['quan_in_city'] = data.city.apply(lambda x: len(data[data.city == x]))

In [ ]:
# data['review_text_tone_coef'] = data['reviews_text'].apply(lambda x: my_functions.review_text_tone(x))

# data['review_text_tone_coef'].value_counts()
# data['good_reviews'] = data.review_text_tone_coef.apply(lambda x: 1 if x > 1 else 0)
# data['bad_reviews'] = data.review_text_tone_coef.apply(lambda x: 1 if x < -1 else 0)


# data['quan_in_city'] = data.city.apply(lambda x: data[data.city == x].size)


# datetime.today().date()

#data.iloc[:2].reviews_dates.apply(max)
# data[data.city == 'London'].ranking_mean_by_city
# data['review_text_tone_coef'].value_counts()
# data['ranking_by_reviews'] = data.ranking_by_reviews.apply(lambda x: (x - data.ranking_by_reviews.mean()) / data.ranking_by_reviews.std())
# data['ranking_by_reviews']
# data['ranking_by_reviews'].value_counts(bins=20)

# EDA

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)

top10_city = data.city.value_counts()[0:10].index
for city in top10_city:
    data[data.city == city].ranking.hist(bins=100, label=city)
    plt.legend()
data.groupby(by='city').number_of_reviews.mean().sort_values()

In [ ]:
data[data.city == 'London'].ranking

In [ ]:
data.city.value_counts(ascending=True).plot(kind='barh')

In [ ]:
data.groupby(by='city').agg({'ranking':'mean', 'rating': 'mean', 'number_of_reviews': 'mean'}).hist()


In [ ]:


data.price_range.value_counts(dropna=False)

# data.groupby(by='city').agg({'price_range_int':'mean', 'city':'size'}).sort_values(by='price_range_int')
# data['price_range'] = data['price_range'].astype(str)
# data.groupby(by='price_range').agg({'city':lambda x: stats.mode(x) , 'ranking': 'mean', 'rating' : 'mean', 'number_of_reviews':'mean', 'dif_days':'mean'})

correlation = data[['rating','ranking',  'number_of_reviews', 'dif_days',
                    #'ranking_by_reviews',
                    'ranking_for_city',
                    'ranking_norm_by_city',
                    'price_range_int',
                    'number_of_reviews_norm',
                    'dif_days'
                    ]].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')

In [ ]:
# data.drop(['number_of_reviews'], inplace=True)

In [ ]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

# Model

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
from sklearn.linear_model import LinearRegression

# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)

X = train_data.drop(['restaurant_id','city', 'city_orig', 'rating', 'cuisine_style', 'price_range', 
               'reviews', 'url_ta', 'id_ta', 'reviews_text',
               #'ranking',
               #'ranking_for_city',
               'number_of_reviews',
               'reviews_dates'
               ], 
               axis = 1, errors='ignore')


y = train_data['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Создаём модель
regr = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(regr.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20900624999999998  
MAE: 0.20914499999999997  
MAE: 0.2097025   
MAE: 0.209354375  
MAE: 0.20936249999999998  
MAE: 0.20936249999999998   
MAE: 0.20969625000000003  
MAE: 0.210050625  
MAE: 0.21450125  
0.21451624999999996  
MAE: 0.21875124999999998  
MAE: 0.2084305  
MAE: 0.210808  
0.2131725  
MAE: 0.2148355  
MAE: 0.21277599999999997  
MAE: 0.2086605   
MAE: 0.34141057440476186

# DATA PREPOCESSING


In [99]:
data = pd.read_csv(input_folder + 'main_task.csv')
data_kaggle = pd.read_csv(input_folder + 'kaggle_task.csv')
sample_submission = pd.read_csv(input_folder + '/sample_submission.csv')

# дря корректной обработки признаков объединяем трейн и тест в один датасет
data['sample'] = 1 # помечаем где у нас трейн
data_kaggle['sample'] = 0 # помечаем где у нас тест
data_kaggle['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = data_kaggle.append(data, sort=False).reset_index(drop=True) # объединяем

In [100]:
preproc_data = my_functions.prepoc_data(data)


In [101]:
preproc_data

,ranking,sample,rating,Number_of_Reviews_isNAN,price_range_int,review_text_tone_coef,dif_days,diff_last_date,Fujian,Portuguese,...,city_orig_Prague,city_orig_Rome,city_orig_Stockholm,city_orig_Vienna,city_orig_Warsaw,city_orig_Zurich,city_orig_nan,ranking_for_city,ranking_norm_by_city,number_of_reviews_norm
0,12963.0,0,0.0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,6855.0,1.491224,-0.420617
1,106.0,0,0.0,0,2,1,37,986,0,0,...,0,0,0,0,0,0,0,-369.0,-1.404530,-0.094045
2,810.0,0,0.0,0,2,0,22,963,0,0,...,0,0,0,0,0,0,0,65.0,-0.146058,-0.336340
3,1669.0,0,0.0,0,3,0,14,985,0,0,...,0,0,0,0,0,0,0,-5524.0,-1.376113,0.274665
4,37.0,0,0.0,0,3,3,15,921,0,0,...,0,0,0,0,0,0,0,-343.0,-1.673120,0.134204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,500.0,1,4.5,0,2,1,34,975,0,0,...,0,0,0,0,0,0,0,-2194.0,-1.448276,-0.157253
49996,6341.0,1,3.5,0,2,0,9,970,0,0,...,0,0,0,0,0,0,0,233.0,-0.147913,1.468583
49997,1652.0,1,4.5,0,2,0,3127,1383,0,0,...,0,0,1,0,0,0,0,619.0,0.573818,-0.420617
49998,641.0,1,4.0,0,2,0,23,1133,0,0,...,0,0,0,0,1,0,0,-284.0,-0.663243,-0.188856


In [104]:
# Теперь выделим тестовую часть
train_data = preproc_data.query('sample == 1').drop(['sample'], axis=1)
test_data = preproc_data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.rating.values            # наш таргет
X = train_data.drop(['rating'], axis=1)

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Model

In [106]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [107]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   31.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


In [108]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20989499999999997


# Submission

In [110]:
test_data = test_data.drop(['rating'], axis=1)

In [111]:
predict_submission = model.predict(test_data)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


In [113]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

,Restaurant_id,Rating
0,id_0,3.050
1,id_1,4.310
2,id_2,4.380
3,id_3,4.290
4,id_4,4.440
5,id_5,4.310
6,id_6,1.340
7,id_7,2.775
8,id_8,4.005
9,id_9,4.695
